# Phase 1: Panel Data Construction & Feature Engineering

**Goal:** To combine all 8 coin datasets into a single, powerful panel data structure and engineer a rich set of features. This notebook will serve as the foundation for our entire project, transforming raw daily data into a clean, feature-rich dataset ready for predictive modeling.

### Task 1.1: Load and Combine Data into a Panel DataFrame

In [1]:
# Core Libraries for Data Handling and Analysis
import numpy as np
import pandas as pd

# Visualization Libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning / Statistics Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# Define the universe of 8 coins for our analysis
coins = [
    "BTCUSD",
    "ETHUSD",
    "XRPUSD",
    "XMRUSD",
    "ZECUSD",
    "TRXUSD",
    "XLMUSD",
    "LTCUSD",
]

In [3]:
# Load the raw data from the CSV file
data_path = "../data/BITFINEX_DATA.csv"
df = pd.read_csv(data_path)

# Filter the dataframe to include only our selected coins
panel_df = df[df["code"].isin(coins)].copy()

# Set a MultiIndex of (date, code) to create a proper panel data structure
# This is crucial for performing panel-aware operations later on
panel_df.set_index(["date", "code"], inplace=True)

In [4]:
# Initial inspection of the panel dataframe
panel_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 25474 entries, ('2014-04-15', 'BTCUSD') to ('2025-07-21', 'XLMUSD')
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   high    25474 non-null  float64
 1   low     25474 non-null  float64
 2   mid     25464 non-null  float64
 3   last    25474 non-null  float64
 4   bid     25464 non-null  float64
 5   ask     25464 non-null  float64
 6   volume  25474 non-null  float64
dtypes: float64(7)
memory usage: 1.6+ MB


In [5]:
# Data Cleaning: In financial data, a value of 0 for price or volume is often an error.
# We replace these with NaN to handle them properly.
panel_df = panel_df.replace(0, np.nan)

# Drop any rows that have NaN in the essential price columns (bid, ask, mid)
# as these are critical for our target variable calculation.
panel_df.dropna(inplace=True)

### Task 1.2: Calculate Base Metrics (Target & Estimators)

Here, we define the functions to calculate our ground-truth target variable (Relative Quoted Spread) and the two microstructure estimators (Corwin-Schultz and Abdi-Ranaldo). These functions are designed to work with `groupby().apply()` to ensure they operate correctly on a per-coin basis.

In [6]:
def calculate_rqs(group: pd.DataFrame) -> pd.Series:
    """
    Computes the Relative Quoted Spread (RQS) for a single asset's DataFrame.
    This is our ground-truth target variable.

    Args:
        group (pd.DataFrame): A DataFrame for a single coin.

    Returns:
        pd.Series: A Series containing the calculated RQS for each day.
    """
    quoted_spread = group["ask"] - group["bid"]
    relative_qs = quoted_spread / group["mid"]
    return relative_qs

In [7]:
def calculate_cs(group: pd.DataFrame) -> pd.Series:
    """
    Computes the Corwin-Schultz spread estimator for a single asset's DataFrame.

    Args:
        group (pd.DataFrame): A DataFrame for a single coin.

    Returns:
        pd.Series: A Series containing the calculated Corwin-Schultz spread.
    """
    # Step 1: Calculate Beta (β) - Sum of squared log high-low ratios over two days
    log_hl_ratio_sq = np.log(group["high"] / group["low"]).pow(2)
    beta = log_hl_ratio_sq.rolling(window=2, min_periods=2).sum()

    # Step 2: Calculate Gamma (γ) - Squared log ratio of the two-day high to the two-day low
    two_day_high = group["high"].rolling(window=2, min_periods=2).max()
    two_day_low = group["low"].rolling(window=2, min_periods=2).min()
    gamma = np.log(two_day_high / two_day_low).pow(2)

    # Step 3: Calculate Alpha (α)
    denominator = 3 - 2 * np.sqrt(2)
    alpha_term1 = (np.sqrt(2 * beta) - np.sqrt(beta)) / denominator
    alpha_term2 = np.sqrt(gamma / denominator)
    alpha = alpha_term1 - alpha_term2

    # Step 4: Calculate the Spread (S)
    spread = 2 * (np.exp(alpha) - 1) / (1 + np.exp(alpha))

    return spread

In [8]:
def calculate_ar(group: pd.DataFrame) -> pd.Series:
    """
    Computes the Abdi-Ranaldo spread estimator for a single asset's DataFrame.

    Args:
        group (pd.DataFrame): A DataFrame for a single coin.

    Returns:
        pd.Series: A Series containing the calculated Abdi-Ranaldo spread.
    """
    # Intermediate calculations based on the paper's methodology
    eta = (np.log(group["high"]) + np.log(group["low"])) / 2
    psi = np.log(group["last"]) - eta

    # Gamma is the product of today's psi and yesterday's psi, capturing price reversal
    gamma = psi * psi.shift(1)

    # The formula requires gamma to be non-negative
    gamma = gamma.clip(lower=0)

    # Final spread calculation
    spread = 2 * np.sqrt(gamma)

    return spread

In [9]:
# Apply the functions to each coin group. `group_keys=False` prevents adding an extra index level.
panel_df["relative_quoted_spread"] = panel_df.groupby("code", group_keys=False).apply(
    calculate_rqs
)
panel_df["corwin_schultz_estimate"] = panel_df.groupby("code", group_keys=False).apply(
    calculate_cs
)
panel_df["abdi_ranaldo_estimate"] = panel_df.groupby("code", group_keys=False).apply(
    calculate_ar
)

# Clean the results: clip any potential negatives that might arise from calculations.
panel_df["corwin_schultz_estimate"] = panel_df["corwin_schultz_estimate"].clip(lower=0)
panel_df["abdi_ranaldo_estimate"] = panel_df["abdi_ranaldo_estimate"].clip(lower=0)

# Drop rows with NaN values that were created by the rolling/shifting operations inside the functions.
panel_df.dropna(
    subset=["corwin_schultz_estimate", "abdi_ranaldo_estimate"], inplace=True
)

# Display the head to verify the new columns.
panel_df.head()

,,high,low,mid,last,bid,ask,volume,relative_quoted_spread,corwin_schultz_estimate,abdi_ranaldo_estimate
date,code,,,,,,,,,,
2014-04-16,BTCUSD,547.0000,495.00,537.500000,538.0000,537.000000,538.00,29633.358705,0.001860,0.000000,0.078938
2014-04-17,BTCUSD,538.5000,486.10,507.020000,508.0000,506.040000,508.00,20709.783819,0.003866,0.060334,0.000000
2014-04-18,BTCUSD,509.0000,474.25,483.770000,482.7500,482.750000,484.79,10458.045243,0.004217,0.000000,0.022383
2014-04-19,BTCUSD,513.9899,473.83,505.010650,507.4999,502.531300,507.49,8963.618369,0.009819,0.063802,0.000000
2014-04-20,BTCUSD,517.9950,492.20,500.745753,501.4400,500.071506,501.42,4921.588803,0.002693,0.016766,0.000000


### Task 1.3: The Bridge - Justifying the PCA Approach

Before we proceed with feature engineering, we must address a critical issue: do our two estimators agree? If they provide conflicting information, using them as raw features could introduce noise into our model. We will visualize their relationship to justify a more sophisticated feature engineering approach.

In [10]:
# Use a longer window (quarterly) to visualize the long-term regime shifts in correlation
window_size = 90

# Create a subplot figure to compare a major coin (BTC) with a more volatile altcoin (TRX)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)

fig.add_trace(
    go.Scatter(
        x=panel_df.xs("BTCUSD", level="code").index,
        y=panel_df.xs("BTCUSD", level="code")["corwin_schultz_estimate"]
        .rolling(window=window_size)
        .corr(panel_df.xs("BTCUSD", level="code")["abdi_ranaldo_estimate"]),
        name="BTCUSD",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=panel_df.xs("TRXUSD", level="code").index,
        y=panel_df.xs("TRXUSD", level="code")["corwin_schultz_estimate"]
        .rolling(window=window_size)
        .corr(panel_df.xs("TRXUSD", level="code")["abdi_ranaldo_estimate"]),
        name="TRXUSD",
    ),
    row=2,
    col=1,
)

fig.update_layout(
    title_text="<b>Rolling Correlation Between CS and AR Estimators (90-Day)</b>",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=600,
)
fig.update_yaxes(title_text="Correlation", range=[-0.8, 0.8])
fig.show()

### The Strategic Solution: From Raw Signals to Engineered Factors via PCA

The plot above reveals a profound insight: **the two estimators are frequently uncorrelated or even negatively correlated for extended periods, and this behavior differs across assets.** This visually confirms that the estimators often diverge, telling conflicting stories about market liquidity.

This presents a clear problem: if we simply include both estimators as separate features in a model, we introduce significant noise and multicollinearity. More importantly, we miss the opportunity to extract the valuable information hidden within their disagreement.

**Instead of viewing this disagreement as noise, we will treat it as a signal in itself.**

To do this, we will employ **Principal Component Analysis (PCA)** on a per-coin basis. PCA is the ideal tool for this situation because it will allow us to decompose the estimators' behavior into two new, more meaningful components:

1.  **Principal Component 1 (PC1) - The "Consensus Stress Factor":** This component will capture the common variance. When both estimators agree that liquidity is poor, this factor will be high. It represents a robust, consensus-based measure of market stress for each coin.

2.  **Principal Component 2 (PC2) - The "Model Disagreement Factor":** This component will capture the orthogonal variance. It will be high when the estimators tell opposite stories, signaling unusual market dynamics.

By constructing these two new factors for each coin, we move from using noisy, raw signals to using sophisticated, engineered features. Our hypothesis is that these factors will be far more powerful predictors of the true `relative_quoted_spread`.

### Task 1.4: Construct PCA Factors (Per-Coin)

In [11]:
def construct_pca_factors(df_coin: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a dataframe for a single coin, standardizes its estimators,
    and returns a dataframe with the two principal components.
    """
    estimators = df_coin[["corwin_schultz_estimate", "abdi_ranaldo_estimate"]]

    if (
        estimators.empty
        or estimators.isnull().all().all()
        or len(estimators.dropna()) < 2
    ):
        return pd.DataFrame(
            {"pc1_stress_factor": pd.NA, "pc2_disagreement_factor": pd.NA},
            index=df_coin.index,
        )

    # 1. Standardize the data (mean=0, std=1) for this coin
    scaler = StandardScaler()
    scaled_estimators = scaler.fit_transform(estimators)

    # 2. Apply PCA
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(scaled_estimators)

    # 3. Create a new dataframe for the results
    pca_df = pd.DataFrame(
        principal_components,
        columns=["pc1_stress_factor", "pc2_disagreement_factor"],
        index=df_coin.index,
    )

    # 4. Check and fix the sign of PC1 for interpretability
    avg_stress = estimators.mean(axis=1)
    if pca_df["pc1_stress_factor"].corr(avg_stress) < 0:
        pca_df["pc1_stress_factor"] *= -1

    return pca_df


# Apply the function to each coin group and join the results back
pca_factors_df = panel_df.groupby("code", group_keys=False).apply(construct_pca_factors)
panel_df = panel_df.join(pca_factors_df)

# --- Verification ---
print("\nVerification: Displaying data for BTCUSD with new PCA factors:")
display(panel_df.loc[(slice(None), "BTCUSD"), :].head())
print(
    f"\nTotal NaNs in new PCA columns: {panel_df[['pc1_stress_factor', 'pc2_disagreement_factor']].isnull().sum().sum()}"
)


Verification: Displaying data for BTCUSD with new PCA factors:


,,high,low,mid,last,bid,ask,volume,relative_quoted_spread,corwin_schultz_estimate,abdi_ranaldo_estimate,pc1_stress_factor,pc2_disagreement_factor
date,code,,,,,,,,,,,,
2014-04-16,BTCUSD,547.0000,495.00,537.500000,538.0000,537.000000,538.00,29633.358705,0.001860,0.000000,0.078938,-3.587426,2.588327
2014-04-17,BTCUSD,538.5000,486.10,507.020000,508.0000,506.040000,508.00,20709.783819,0.003866,0.060334,0.000000,2.423475,1.582815
2014-04-18,BTCUSD,509.0000,474.25,483.770000,482.7500,482.750000,484.79,10458.045243,0.004217,0.000000,0.022383,-1.073977,0.074878
2014-04-19,BTCUSD,513.9899,473.83,505.010650,507.4999,502.531300,507.49,8963.618369,0.009819,0.063802,0.000000,2.567329,1.726670
2014-04-20,BTCUSD,517.9950,492.20,500.745753,501.4400,500.071506,501.42,4921.588803,0.002693,0.016766,0.000000,0.616231,-0.224428



Total NaNs in new PCA columns: 0


### Task 1.5: Engineer Standard Market Features (Panel-Aware)

Now we engineer a set of standard market features related to momentum, volatility, and volume. All calculations are performed using `groupby('code')` to ensure they are panel-aware.

In [12]:
# Daily log returns
panel_df["log_return"] = panel_df.groupby("code")["last"].transform(
    lambda x: np.log(x / x.shift(1))
)
# Rolling sum of log returns for longer-term momentum
panel_df["log_return_5d"] = panel_df.groupby("code")["log_return"].transform(
    lambda x: x.rolling(window=5).sum()
)
panel_df["log_return_21d"] = panel_df.groupby("code")["log_return"].transform(
    lambda x: x.rolling(window=21).sum()
)

# Simple High-Low Range Volatility
panel_df["volatility_hl"] = (panel_df["high"] - panel_df["low"]) / panel_df["low"]
# Historical Volatility (Standard Deviation of Log Returns)
panel_df["volatility_21d"] = panel_df.groupby("code")["log_return"].transform(
    lambda x: x.rolling(window=21).std()
)

# USD Volume for comparability
panel_df["volume_usd"] = panel_df["volume"] * panel_df["mid"]
# Volume Change (as a log ratio to a smoothed average)
rolling_avg_volume = panel_df.groupby("code")["volume_usd"].transform(
    lambda x: x.rolling(window=5).mean()
)
panel_df["volume_change_ratio"] = np.log(panel_df["volume_usd"] / rolling_avg_volume)

# 50-day Simple Moving Average (SMA)
panel_df["sma_50d"] = panel_df.groupby("code")["last"].transform(
    lambda x: x.rolling(window=50).mean()
)
# Binary feature: 1 if price is above SMA, 0 if below
panel_df["trend_above_sma50"] = (panel_df["last"] > panel_df["sma_50d"]).astype(int)

In [13]:
# Save panel_df into a csv file. We'll use it in backtesting part later
panel_df.to_csv("../data/panel_df.csv")

### Task 1.6: Create Lagged Features & Finalize Phase 1

This is the final and most critical step of data preparation. We create lagged versions of our features to ensure that our model only uses past information to predict future outcomes, thus avoiding lookahead bias. After lagging, we drop all rows containing any NaN values to produce our final, clean dataset for modeling.

In [14]:
# Define all the columns we want to create lagged versions of
features_to_lag = [
    "relative_quoted_spread",  # Lagging the target itself is a powerful autoregressive feature
    "corwin_schultz_estimate",
    "abdi_ranaldo_estimate",
    "pc1_stress_factor",
    "pc2_disagreement_factor",
    "log_return",
    "log_return_5d",
    "log_return_21d",
    "volatility_hl",
    "volatility_21d",
    "volume_change_ratio",
    "trend_above_sma50",
]

# Define the number of lags (1 to 5 days captures a full trading week)
n_lags = 5

# Loop and create lagged features using the panel-aware groupby().shift()
for i in range(1, n_lags + 1):
    for feature in features_to_lag:
        new_col_name = f"{feature}_lag_{i}"
        panel_df[new_col_name] = panel_df.groupby("code")[feature].shift(i)

# Final Data Cleaning
print(f"\nShape of dataframe before dropping NaNs: {panel_df.shape}")

# Drop all rows that have at least one NaN value to get the final model-ready dataset
final_model_df = panel_df.dropna()

print(f"Shape of dataframe after dropping NaNs: {final_model_df.shape}")

# Verification
print(f"Total NaNs in final dataframe: {final_model_df.isnull().sum().sum()}")


Shape of dataframe before dropping NaNs: (25449, 81)
Shape of dataframe after dropping NaNs: (25057, 81)
Total NaNs in final dataframe: 0


### Task 1.7: Save the final dataframe

In [15]:
# Save final_model_df into a csv file in data folder
target_path = "../data/final_model_df.csv"
final_model_df.to_csv(target_path)

### Conclusion of Phase 1

Phase 1 is complete. We have successfully transformed raw, multi-asset time-series data into a clean, feature-rich panel dataset named `final_model_df`. This dataframe is now ready for the predictive modeling tasks in Phase 2.